<a href="https://colab.research.google.com/github/davidlealo/aprendiendolangchain/blob/main/libros_extensos_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalación de bibliotecas necesarias
!pip install langchain openai PyPDF2 faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 16.5 MB/s eta 0:00:00


In [ ]:
# Librerías necesarias
from google.colab import files
import getpass
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Subir la API Key de OpenAI
api_key = getpass.getpass("Por favor, introduce tu API key de OpenAI: ")
import os
os.environ["OPENAI_API_KEY"] = api_key

# Subir archivo PDF
print("Sube el libro en formato PDF:")
uploaded = files.upload()

# Verificar archivo subido
file_name = list(uploaded.keys())[0]
print(f"Archivo cargado: {file_name}")

# Cargar el libro
loader = PyPDFLoader(file_name)
pages = loader.load()

# Dividir el libro en fragmentos
print("Dividiendo el libro en fragmentos manejables...")
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = splitter.split_documents(pages)
print(f"Se dividió el libro en {len(documents)} fragmentos.")

# Generar embeddings para los fragmentos
print("Generando embeddings...")
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts([doc.page_content for doc in documents], embeddings)

# Guardar la base de datos vectorial en la memoria
vectorstore.save_local("vectorstore_faiss")
print("Base de datos vectorial creada y almacenada.")

# Crear la cadena de preguntas y respuestas
print("Configurando el sistema de consultas...")
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(model="gpt-4"),
    retriever=vectorstore.as_retriever(),
)

# Preguntar al usuario
print("\nAhora puedes hacer preguntas sobre el libro.")
while True:
    pregunta = input("\nEscribe tu pregunta (o 'salir' para terminar): ")
    if pregunta.lower() == "salir":
        print("¡Gracias por usar el analizador de libros!")
        break
    respuesta = qa_chain.run(pregunta)
    print(f"\nRespuesta:\n{respuesta}")

# Análisis avanzado (resúmenes automáticos)
def generar_resumen():
    print("\nGenerando un resumen del libro...")
    prompt = PromptTemplate(
        input_variables=["texto"],
        template="Por favor, resume el siguiente texto:\n\n{texto}",
    )
    summary_chain = LLMChain(llm=OpenAI(model="gpt-4"), prompt=prompt)
    resumen = summary_chain.run(documents[0].page_content)  # Resumir el primer fragmento
    print(f"\nResumen del primer fragmento:\n{resumen}")

# Opcional: Resúmenes
generar_resumen()
